# 自定义类

## 私有属性和方法

Python没有为私有属性提供直接的支持，然而，通过玩点小花招，可获得类似于私有属性的效果。

要让方法或属性成为私有的（不能从外部访问），只需让*其名称以两个下划线打头*即可。

在幕后，python会对所有以两个下划线打头的名称都进行转换，即在开头加上一个下划线和类名。只要知道这种幕后处理手法，就能从类外访问私有方法，然而不应这样做。总之，你无法禁止别人访问对象的私有方法和属性，但这种名称修改方式发出了强烈的信号，让他们不要这样做。

如果你不希望名称被修改，又想发出不要从外部修改属性或方法的信号，*可用一个下划线打头*。这虽然只是一种约定，但也有些作用。例如， `from module import *`不会导入以一个下划线打头的名称。

In [1]:
class Secretive:

    def __inaccessible(self):
        print("Bet you can't see me ...")

    
    def _ignore_me(self):
        print("you should not call me outside.")

    
    def accessible(self):
        print("The secret message is:")
        self.__inaccessible()

In [2]:
s = Secretive()
s.accessible()
s._ignore_me()  # 不要这么做

The secret message is:
Bet you can't see me ...
you should not call me outside.


In [3]:
# s.__inaccessible()
s._Secretive__inaccessible()  # 在幕后，python会对所有以两个下划线打头的名称都进行转换，即在开头加上一个下划线和类名

Bet you can't see me ...


## 类命名空间

类定义其实就是创造一个命名空间，将相关的代码放到这个命名空间下，即类的命名空间。基于类创建的实例，又创建了一个命名空间，关联实例所特有的属性，即实例的命名空间。

类定义中，并非只能包含def语句。

In [4]:
class C:
    print('Class C being defined...')

Class C being defined...


In [5]:
class MemberCounter:
    members = 0  # 在类命名空间定义的变量，所有的实例都可以访问
    def init(self):
        MemberCounter.members += 1

In [6]:
m1 = MemberCounter()
m1.init()
MemberCounter.members

1

In [7]:
m2 = MemberCounter()
m2.init()
MemberCounter.members

2

In [8]:
m1.members, m2.members

(2, 2)

In [9]:
m1.members = 'Two'  # 这样会在实例命名空间下定义同名变量，并遮盖了类变量
m1.members, m2.members

('Two', 2)

## 指定超类

In [10]:
class Filter:
    def init(self):
        self.blocked = []
    def filter(self, sequence):
        return [x for x in sequence if x not in self.blocked]

class SPAMFilter(Filter):  # 括号内指定超类
    def init(self):  # 重写init方法
        self.blocked = ['SPAM']

In [11]:
f = Filter()
f.init()
f.filter([1, 2, 3])

[1, 2, 3]

In [12]:
s = SPAMFilter()
s.init()
s.filter(['SPAM', 'SPAM', 'SPAM', 'SPAM', 'eggs', 'bacon', 'SPAM'])

['eggs', 'bacon']

In [13]:
# issubclass: 判断子类关系
issubclass(SPAMFilter, Filter)

True

In [14]:
# isinstance: 判断实例关系
isinstance(s, SPAMFilter), isinstance(s, Filter)

(True, True)

In [15]:
# __bases__返回类的基类
SPAMFilter.__bases__

(__main__.Filter,)

In [16]:
# __class__返回实例的类
s.__class__

__main__.SPAMFilter

In [17]:
# python支持多继承

class Calculator:
    def calculate(self, expression):
        self.value = eval(expression)

class Talker:
    def talk(self):
        print('Hi, my value is', self.value)

"""
应尽量避免使用多继承，需要注意括号内超类的顺序对同名方法的覆盖关系有影响。
"""

class TalkingCalculator(Calculator, Talker):
    pass

In [18]:
tc = TalkingCalculator()
tc.calculate('1 + 2 * 3')
tc.talk()

Hi, my value is 7


## 接口和内省

我们编写的自定义类或函数是为了在某个地方进行应用，代码不是封闭的，上层代码一般需要提供输入给目标类，并从目标类中获取对应的结果。

在我们编写的类内部，一般对传入的参数/输入有一些期望，比如参数是什么类型？需要提供什么方法？等。

在Python的世界里，我们有2种方式：1种是相信上层代码知道该传什么，直接按照预期的方式去使用，比如直接调用 arg.method_a()。如果参数不满足要求，允许程序直接失败；第2种是我们尽量检查传入的参数是否符合要求，只有符合要求的情况下才会调用。

标题中的“接口”，即要求对象实现特定的方法，或提供特定的熟悉；“内省”，即可通过一定的手段对其进行检查。

p.s. 第1种方式又称为鸭子类型，即假设所有对象都能完成其工作。

In [19]:
# hasattr: 检查对象具备某个属性
hasattr(tc, 'talk')

True

In [20]:
hasattr(tc, "fnord")

False

In [21]:
# getattr: 获取对应的属性
# callable: 检查是否可调用

callable(getattr(tc, 'talk', None))  # getattr的第3个参数：表示属性不存在时返回None

True

In [22]:
callable(getattr(tc, 'fnord', None))

False

In [23]:
# setattr: 设置对象的属性
setattr(tc, 'name', 'Mr. Gumby')
tc.name

'Mr. Gumby'

In [24]:
# __dict__ 可用于查看对象中存储的所有值
tc.__dict__

{'value': 7, 'name': 'Mr. Gumby'}

## 抽象基类

抽象基类可以定义抽象的方法，并要求子类实现，如果不实现将无法实例化。

抽象基类可用于针对应满足的协议进行定义：即目标类应该定义某些方法。这样我们可以不必一一检查每一个属性/方法是否存在，而是通过检查类实例是否属于该抽象基类的实例即可。

对于历史的代码，我们可以通过注册的方式，声明它属于新定义的抽象基类，但是需要用户自己确保它满足基类所有的协议。

In [25]:
# abc模块提供了这种机制的实现

from abc import ABC, abstractclassmethod

class Talker(ABC):

    @abstractclassmethod
    def talk(self):
        pass

In [26]:
# TypeError: Can't instantiate abstract class Talker with abstract methods talk

Talker()

TypeError: Can't instantiate abstract class Talker with abstract method talk

In [27]:
# 派生子类，实现抽象的方法，则可以实例化该子类

class Knigget(Talker):
    def talk(self):
        print("Ni!")

k = Knigget()
k.talk()
isinstance(k, Talker)  # 通过接口类型检查，可以确定实例实现了接口的所有的方法/协议，避免一个个检查

Ni!


True

In [28]:
# 可以将非继承关系包装成继承关系，在兼容历史代码时可能有用

class Herring:
    def talk(self):
        print("Blub.")

h = Herring()
h.talk()
isinstance(h, Talker)  # False

Blub.


False

In [29]:
# 包装的关键是使用 register 方法
Talker.register(Herring)

isinstance(h, Talker)  # True

True

In [30]:
# 包装的方法，需要用户自行确保协议安全性。从抽象类提供的保障将不存在
class Clam:
    pass

Talker.register(Clam)

c = Clam()
isinstance(c, Talker)  # True

c.talk()  # AttributeError: 'Clam' object has no attribute 'talk'

AttributeError: 'Clam' object has no attribute 'talk'